In [1]:
import requests
import matplotlib.pyplot as plt
import json
import pandas as pd

In [2]:
def url_to_df(url):
    endpoint = url
    response = requests.get(endpoint)
    response
    if response.status_code == 200:
        F1 = response.json()
        F1_df = pd.DataFrame(F1)
    else: print(response.status_code)
    return F1_df

In [3]:
def url_to_df_in_chunks(base_url, max_pages=None, params=None):
    all_data = []
    page = 1
    params = params or {}  # default to empty dict if None

    while True:
        params.update({'start': page})  # or 'offset'/'start', depending on API
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()

            # Stop if empty (no more pages)
            if not data:
                break

            all_data.extend(data)  # Append this page’s data

            page += 1
            if max_pages and page > max_pages:
                break
        except requests.exceptions.RequestException as err:
            print(f"Request failed on page {page}: {err}")
            break
        except ValueError as json_err:
            print(f"JSON error on page {page}: {json_err}")
            break

    return pd.DataFrame(all_data)

In [ ]:
monaco23 = url_to_df('https://api.openf1.org/v1/car_data?session_key=9094&driver_number=4')
monaco23

In [ ]:
monaco23['date'] = pd.to_datetime(monaco23['date'],format='ISO8601')
monaco23['date'] = monaco23['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
monaco23 = monaco23[monaco23.date > '2023-05-28 13:01:0000+00:00']
monaco23 = monaco23[monaco23.date < '2023-05-28 14:53:0000+00:00']
monaco23

In [ ]:
lando_laps = pd.read_csv('../data/laps_lando_api.csv')
lando_laps = lando_laps[['session_key','date_start','driver_number','lap_number']]
lando_laps = lando_laps.rename(columns = {'date_start':'date'})
lando_laps = lando_laps[lando_laps.session_key == 9094]
lando_laps

In [ ]:
monaco = monaco23.merge(lando_laps, on = ['session_key','date','driver_number'], how='left')
monaco.head(60)

In [ ]:
monaco = monaco.drop(columns = 'meeting_key')
monaco

In [ ]:
monaco_loc = url_to_df('https://api.openf1.org/v1/location?session_key=9094&driver_number=4')
monaco_loc

In [ ]:
monaco_loc['date'] = pd.to_datetime(monaco_loc['date'],format='ISO8601')
monaco_loc['date'] = monaco_loc['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
monaco_loc = monaco_loc.drop(columns = ['z','meeting_key'])
monaco_loc

In [ ]:
monaco_loc.to_csv('../data/clean/mon_loc.csv')

In [ ]:
monaco.to_csv('../data/clean/monaco1.csv')